## Loading Reflectance

In [3]:
import array
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
from termcolor import colored
import time
import xarray as xr

**Functions**

In [4]:
nd = -28672

In [5]:
def fromstring(x):
    
    '''Converts string to datetime object'''
    
    try:
        d = datetime.datetime.strptime(x, '%d/%m/%Y').date()
    except:
        d = datetime.datetime.strptime(x, '%Y-%m-%d').date()
        
    return d

def fromjulian(x):
    
    '''Converts julian to datetime object'''

    return datetime.datetime.strptime(x, '%Y%j').date()

def compute_ndvi_float(b1: float, b2:float):
    
    if ((b2 + b1) == 0):
        ndvi = 0
    else:
        ndvi = (b2 - b1)/(b2 + b1)
        
    return ndvi

**Loading**

In [15]:
# loading MOD data from csv
MOD = pd.read_csv(
    'data/MOD09A1-MOD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MOD09A1_006_sur_refl_b01', 
               'MOD09A1_006_sur_refl_b02',
               'MOD09A1_006_sur_refl_b03',
               'MOD09A1_006_sur_refl_b04',
               'MOD09A1_006_sur_refl_b05',
               'MOD09A1_006_sur_refl_b06',
               'MOD09A1_006_sur_refl_b07',
               'MOD09A1_006_sur_refl_day_of_year'],
     dtype = {'MOD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
MOD = MOD.rename(columns={"MOD09A1_006_sur_refl_b01": "B1", 
                        "MOD09A1_006_sur_refl_b02": "B2",
                        "MOD09A1_006_sur_refl_b03": "B3",
                        "MOD09A1_006_sur_refl_b04": "B4",
                        "MOD09A1_006_sur_refl_b05": "B5",
                        "MOD09A1_006_sur_refl_b06": "B6",
                        "MOD09A1_006_sur_refl_b07": "B7",
                        'MOD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
MOD['Date'] = MOD['Date'].apply(fromstring)

#Cropping to right dates
a = (MOD['Date']>datetime.date(2002,7,4)).values
MOD = MOD[a]

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(MOD)):
    
    d = MOD['Date'][i]
    c = MOD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
MOD['Composite_date'] = compo   
MOD = MOD[same_compo]

In [13]:
# loading MYD data from csv
MYD = pd.read_csv(
    'data/MYD09A1-MYD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Date', 
               'MYD09A1_006_sur_refl_b01', 
               'MYD09A1_006_sur_refl_b02',
               'MYD09A1_006_sur_refl_b03',
               'MYD09A1_006_sur_refl_b04',
               'MYD09A1_006_sur_refl_b05',
               'MYD09A1_006_sur_refl_b06',
               'MYD09A1_006_sur_refl_b07',
               'MYD09A1_006_sur_refl_day_of_year'],
     dtype = {'MYD09A1_006_sur_refl_day_of_year': int})


#renaming the columns
MYD = MYD.rename(columns={"MYD09A1_006_sur_refl_b01": "B1", 
                        "MYD09A1_006_sur_refl_b02": "B2",
                        "MYD09A1_006_sur_refl_b03": "B3",
                        "MYD09A1_006_sur_refl_b04": "B4",
                        "MYD09A1_006_sur_refl_b05": "B5",
                        "MYD09A1_006_sur_refl_b06": "B6",
                        "MYD09A1_006_sur_refl_b07": "B7",
                        'MYD09A1_006_sur_refl_day_of_year': 'Composite_date'})


# Convert string Date to datetime.date
MYD['Date'] = MYD['Date'].apply(fromstring)

# Convert composite_date from julian to datetime
# Add a True/False column to keep track of the values with the same composite_date

compo = []
same_compo = []

for i in range(len(MYD)):
    
    d = MYD['Date'][i]
    c = MYD['Composite_date'][i]
    
    if c != 65535:    #because nd is 65535    
        if (d.month == 12) and (c<20):
            compo.append(fromjulian(str(d.year+1)+str(c)))
        else:
            compo.append(fromjulian(str(d.year)+str(c)))
    else:
        # nodata so we don't care about the date
        compo.append(d)
    
    if (i==0):
        same_compo.append(True)
    elif (compo[i]==compo[i-1]):
        same_compo.append(False)
    else:
        same_compo.append(True)
            
MYD['Composite_date'] = compo   
MYD = MYD[same_compo]

In [17]:
MYD.index

Index(['AFG01a', 'AFG01a', 'AFG01a', 'AFG01a', 'AFG01a', 'AFG01a', 'AFG01a',
       'AFG01a', 'AFG01a', 'AFG01a',
       ...
       'ZAM14b', 'ZAM14b', 'ZAM14b', 'ZAM14b', 'ZAM14b', 'ZAM14b', 'ZAM14b',
       'ZAM14b', 'ZAM14b', 'ZAM14b'],
      dtype='object', name='ID', length=147030)

In [19]:
# Creating MXD

MXD = pd.DataFrame()

# Blend MOD and MYD as function of composite_date for each location
locations = sorted(list(set(MYD.index)))
for l in locations:
    
    MOD_loc = MOD.loc[l]
    MOD_loc = MOD_loc.reset_index()
    MOD_loc = MOD_loc.set_index('Composite_date')
    
    MYD_loc = MYD.loc[l]
    MYD_loc = MYD_loc.reset_index()
    MYD_loc = MYD_loc.set_index('Composite_date')
    
    MXD_loc = pd.concat([MOD_loc, MYD_loc]).sort_index()
    MXD_loc = MXD_loc.reset_index()
    MXD_loc = MXD_loc.set_index('ID')
    
    MXD = pd.concat([MXD, MXD_loc])

# Remove values with same composite_date (keep max value)
same_compo = []
for i in range(len(MXD)):    
    if (i==0):
        same_compo.append(True)
    elif (MXD['Composite_date'][i]==MXD['Composite_date'][i-1]):
        if compute_ndvi_float(MXD['B1'][i-1],MXD['B2'][i-1]) > compute_ndvi_float(MXD['B1'][i],MXD['B2'][i]): 
            same_compo.pop()
            same_compo.append(False)
            same_compo.append(True)
        else:
            same_compo.append(False)
    else:
        same_compo.append(True)
        
MXD = MXD[same_compo]

In [20]:
#Creating lat lon dictionnary

latlon = {}

pd_latlon = pd.read_csv(
    'data/MOD09A1-MOD09A1-006-results.csv', 
    index_col=0, 
    usecols = ['ID', 
               'Latitude', 
               'Longitude'])

dict_index = set(pd_latlon.index)

for location in dict_index:
    latlon[location] = (round(pd_latlon.loc[location]['Latitude'][0],3), round(pd_latlon.loc[location]['Longitude'][0],3))